In [33]:
# TODO
# Пофиксить артефакты 'chrNT_113889.1', 'chrNT_187633.1', 'chrNT_187661.1', 'chrNT_187693.1', 'chrNW_009646201.1'

In [2]:
import csv
import gzip
import os

In [30]:
headers = ['Chr', 'Position', 'rsID', 'Ref', 'Alt', 'Classification']

In [22]:
clinvar_columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO']
clinvar_field_mapping = dict(zip(clinvar_columns, range(len(clinvar_columns))))

In [23]:
clinvar_field_mapping

{'CHROM': 0,
 'POS': 1,
 'ID': 2,
 'REF': 3,
 'ALT': 4,
 'QUAL': 5,
 'FILTER': 6,
 'INFO': 7}

In [4]:
clinvar_info_names = 'AF_ESP|AF_EXAC|AF_TGP|ALLELEID|CLNDN|CLNDNINCL|CLNDISDB|CLNDISDBINCL|CLNHGVS|CLNREVSTAT|CLNSIG|CLNSIGCONF|CLNSIGINCL|CLNVC|CLNVCSO|CLNVI|DBVARID|GENEINFO|MC|ONCDN|ONCDNINCL|ONCDISDB|ONCDISDBINCL|ONC|ONCINCL|ONCREVSTAT|ONCCONF|ORIGIN|RS|SCIDN|SCIDNINCL|SCIDISDB|SCIDISDBINCL|SCIREVSTAT|SCI|SCIINCL'.split('|')

Значения колонки CHROM:  
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, '20', '21', '22', 'X', 'Y', 'MT', 'NT_113889.1', 'NT_187633.1',
       'NT_187661.1', 'NT_187693.1', 'NW_009646201.1']

Я прибавляю _chr_ к значениям хромосом, чтобы было единообразие с результатами парсинга гномада. Т.е. со значениями типа NT_113889.1 получается некрасивое chrNT_113889.1

In [24]:
# Функция для извлечения данных из поля INFO Clinvar

def get_clinvar_info(line, target_field):
    info_index = clinvar_field_mapping['INFO']
    clinvar_info = line[info_index].split(';')  # разделяем все поля по ;

    for i in clinvar_info:
        value = None
        if i.startswith(target_field):
            i = i.split('=')
            value = i[-1]

            return value

In [31]:
# Функция для парсинга вцф-файла

def get_parse_clinvar_vcf(vcf_file, output_dir='', output_filename=''):

    # Если папка не указана, сохраняем в текущую директорию
    if output_dir == '':
        output_dir = os.getcwd()
    
    # Если имя выходного файла не задано, используем имя входного файла, но с расширением .tsv
    # Через basename получаем имя файла, а через splitext оставляем только название без расширения
    if output_filename == '':
        output_filename = os.path.splitext(os.path.basename(vcf_file))[0] + '.tsv'
    else:
        # Если не указано расширение файла (указано неверно), добавляем .tsv
        if not output_filename.endswith('.tsv'):
            output_filename += '.tsv'
    
    # Полный путь к выходному файлу
    output_file = os.path.join(output_dir, output_filename)
    
    # Запись шапки таблицы в выходной файл
    with open(output_file, 'w', newline='', encoding='utf-8') as table_file:
        writer = csv.writer(table_file, delimiter='\t')
        writer.writerow(headers)

    # Открываем на чтение файл, который надо распарсить
    with gzip.open(vcf_file, 'rt') as input_file, open(output_file, 'a', newline='', encoding='utf-8') as output_file:
        vcf_reader = csv.reader(input_file, delimiter='\t')
        tsv_writer = csv.writer(output_file, delimiter='\t')

        for line in vcf_reader:
            if not line[clinvar_field_mapping['CHROM']].startswith('#'):
                    
                    chrom = 'chr' + line[clinvar_field_mapping['CHROM']]
                    position = line[clinvar_field_mapping['POS']]
                    rsID = line[clinvar_field_mapping['ID']]
                    Ref = line[clinvar_field_mapping['REF']]
                    Alt = line[clinvar_field_mapping['ALT']]
                    
                    significance = get_clinvar_info(line, 'CLNSIG')

                    # Записываем новые данные в таблицу
                    filtered_data = [
                                    chrom, 
                                    position, 
                                    rsID, 
                                    Ref, 
                                    Alt,
                                    significance
                                ]

                    tsv_writer.writerow(filtered_data)

Указываем input файл (или путь к нему), а также имя output (необязательно, задаётся по умолчанию по названию вцф):

In [32]:
get_parse_clinvar_vcf('data_dir/clinvar/clinvar_20240331.vcf.gz', output_dir='data_dir')